In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
jarv_csv="/home/akis-linardos/Datasets/OPTIMAM/jarv_info.csv"
stge_csv="/home/akis-linardos/Datasets/OPTIMAM/stge_info.csv"
inbreast_csv="/home/lidia-garrucho/datasets/INBREAST/INbreast_updated_cropped_breast.csv"
cmmd_csv='/home/akis-linardos/Datasets/CMMD/info.csv'
bcdr_d01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D01_dataset/dataset_info.csv"
bcdr_d02_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D02_dataset/dataset_info.csv"
bcdr_dn01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-DN01_dataset/dataset_info.csv"

b_d01_csv=pd.read_csv(bcdr_d01_csv)
b_d02_csv=pd.read_csv(bcdr_d02_csv)
b_dn01_csv=pd.read_csv(bcdr_dn01_csv)
j_csv=pd.read_csv(jarv_csv)
s_csv=pd.read_csv(stge_csv)
i_csv=pd.read_csv(inbreast_csv)
c_csv=pd.read_csv(cmmd_csv)

In [104]:
b_d01_csv['density'].max()

4

In [17]:
jarv = j_csv[['laterality','status']]
stge = s_csv[['laterality','status']]
inbr = i_csv[['laterality','BIRADS']]
bcd1 = b_d01_csv[['laterality', 'classification']]
bcd2 = b_d02_csv[['laterality', 'classification']]
cmmd = c_csv[['laterality','status']]

In [20]:
jarv.iloc[0]

laterality            L
status        Malignant
Name: 0, dtype: object

In [21]:
stge.iloc[0]

laterality         R
status        Benign
Name: 0, dtype: object

In [109]:
set(s_csv['manufacturer'])

{'GE MEDICAL SYSTEMS', 'HOLOGIC, Inc.', 'Philips Medical Systems', 'SIEMENS'}

## Laterality

In [82]:
def laterality_stats(dataset_pd, label='status'):
    r_benign, r_malign, l_benign, l_malign = 0, 0, 0, 0
    for row in dataset_pd.iterrows():
        if (row[1][label]=='Normal' or row[1][label]=='Interval Cancer'):
            continue

        if (row[1]['laterality']=='LEFT' or row[1]['laterality']=='L') and (row[1][label]=='Malignant' or row[1][label]=='Malign'):
            l_malign += 1
        elif (row[1]['laterality']=='RIGHT' or row[1]['laterality']=='R') and (row[1][label]=='Malignant' or row[1][label]=='Malign'):
            r_malign += 1
        elif (row[1]['laterality']=='LEFT' or row[1]['laterality']=='L') and (row[1][label]=='Benign' or row[1][label]=='Benign'):
            l_benign += 1
        elif (row[1]['laterality']=='RIGHT' or row[1]['laterality']=='R') and (row[1][label]=='Benign' or row[1][label]=='Benign'):
            r_benign += 1
        else:
            print("ERROR One of the two not recognized: {} {}".format(row[1]['laterality'], row[1][label]))
            break
    ml = l_malign/(l_malign+r_malign)
    mr = r_malign/(l_malign+r_malign)
    bl = l_benign/(l_benign+r_benign)
    br = r_benign/(l_benign+r_benign)
    return [ml, mr, bl, br]

In [79]:
def laterality_stats_inbreast(dataset_pd, label='BIRADS'):
    r_benign, r_malign, l_benign, l_malign = 0, 0, 0, 0
    for row in dataset_pd.iterrows():
        if (row[1][label]=='1'):
            continue

        if (row[1]['laterality']=='LEFT' or row[1]['laterality']=='L') and (row[1][label] == '4' or row[1][label]== '5' or row[1][label]=='6'):
            l_malign += 1
        elif (row[1]['laterality']=='RIGHT' or row[1]['laterality']=='R') and (row[1][label] == '4' or row[1][label]== '5' or row[1][label]=='6'):
            r_malign += 1
        elif (row[1]['laterality']=='LEFT' or row[1]['laterality']=='L') and (row[1][label]=='2' or row[1][label]=='3'):
            l_benign += 1
        elif (row[1]['laterality']=='RIGHT' or row[1]['laterality']=='R') and (row[1][label]=='2' or row[1][label]=='3'):
            r_benign += 1
        else:
            print("ERROR One of the two not recognized: {} {}".format(row[1]['laterality'], row[1][label]))
            break
    ml = l_malign/(l_malign+r_malign)
    mr = r_malign/(l_malign+r_malign)
    bl = l_benign/(l_benign+r_benign)
    br = r_benign/(l_benign+r_benign)
    return [ml, mr, bl, br]

In [43]:
laterality_stats(stge)

(0.32525435811029746,
 0.32693927807659906,
 0.17380597498541897,
 0.17400038882768454)

In [63]:
laterality_stats_inbreast(inbr, label='BIRADS')


ERROR One of the two not recognized: R 4a


[0.0, 0.2857142857142857, 0.7142857142857143, 0.0]

In [83]:
laterality_DF_stats = pd.DataFrame(np.array([laterality_stats(stge), laterality_stats(jarv), laterality_stats_inbreast(inbr, label='BIRADS')
, laterality_stats(bcd1, 'classification'), laterality_stats(bcd2, 'classification'), laterality_stats(cmmd)]), columns=['Left+Malignant', 'Right+Malignant', 'Left+Benign', 'Right+Benign'], index=['jarv', 'stge', 'inbr', 'bcd1', 'bcd2', 'cmmd'])

ERROR One of the two not recognized: R 4a


In [84]:
laterality_DF_stats

,Left+Malignant,Right+Malignant,Left+Benign,Right+Benign
jarv,0.498708,0.501292,0.499721,0.500279
stge,0.500385,0.499615,0.499166,0.500834
inbr,0.000000,1.000000,1.000000,0.000000
bcd1,0.666667,0.333333,0.453488,0.546512
bcd2,0.700000,0.300000,0.553086,0.446914
cmmd,0.539364,0.460636,0.642599,0.357401


In [75]:
bcdr = pd.concat([bcd1, bcd2])

In [85]:
laterality_DF_stats = pd.DataFrame(np.array([laterality_stats(stge), laterality_stats(jarv), laterality_stats_inbreast(inbr, label='BIRADS')
, laterality_stats(bcdr, 'classification'), laterality_stats(cmmd)]), columns=['Left+Malignant', 'Right+Malignant', 'Left+Benign', 'Right+Benign'], index=['OPTIMAM(jarv)', 'OPTIMAM(stge)', 'InBreast', 'BCDR', 'CMMD'])

ERROR One of the two not recognized: R 4a


In [86]:
laterality_DF_stats

,Left+Malignant,Right+Malignant,Left+Benign,Right+Benign
OPTIMAM(jarv),0.498708,0.501292,0.499721,0.500279
OPTIMAM(stge),0.500385,0.499615,0.499166,0.500834
InBreast,0.000000,1.000000,1.000000,0.000000
BCDR,0.682243,0.317757,0.535642,0.464358
CMMD,0.539364,0.460636,0.642599,0.357401


In [ ]:
df2 = pd.DataFrame(np.array([laterality_stats(stge)]), columns=[])